In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models , transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import torchvision
import numpy as np


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)



Using device: cuda


In [3]:
image_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2,contrast=0.2),
    transforms.RandomRotation(10),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225))]    
)

In [4]:
dataset_path = dataset_path = r"C:\Data_Learnings\Internship2\FreshHarvest_Dataset\FRUIT-16K"
dataset = datasets.ImageFolder(dataset_path,transform=image_transform)
len(dataset)

16000

In [5]:
dataset.classes

['F_Banana',
 'F_Lemon',
 'F_Lulo',
 'F_Mango',
 'F_Orange',
 'F_Strawberry',
 'F_Tamarillo',
 'F_Tomato',
 'S_Banana',
 'S_Lemon',
 'S_Lulo',
 'S_Mango',
 'S_Orange',
 'S_Strawberry',
 'S_Tamarillo',
 'S_Tomato']

In [7]:
num_classes = len(dataset.classes)
num_classes

16

In [8]:
train_size = int(0.8 * len(dataset))
train_size

12800

In [9]:
val_size = int(len(dataset) - train_size)
val_size

3200

In [ ]:
from torch.utils.data import random_split
train_dataset, val_dataset = random_split(dataset, [train_size, val_size]) # splitting the image dataset into training and validataion dataset

In [11]:
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True) # saving the images in train_loader
val_loader = DataLoader(val_dataset,batch_size=32,shuffle=True) 

In [ ]:
for images,labels in train_loader:
    print(images.shape) # torch.Size([32, 3, 224, 224]) → batch of 32 images, each with 3 color channels and size 224x224
    print(labels.shape)  # torch.Size([32]) → batch of 32 labels
    break

torch.Size([32, 3, 224, 224])
torch.Size([32])


In [15]:
images[31],labels[31]


(tensor([[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],
 
         [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],
 
         [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
          [-1.8044, -1.8044,